In [ ]:
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install datasets

In [ ]:
!pip show peft

Name: peft
Version: 0.5.0.dev0
Summary: Parameter-Efficient Fine-Tuning (PEFT)
Home-page: https://github.com/huggingface/peft
Author: The HuggingFace team
Author-email: sourab@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: accelerate, numpy, packaging, psutil, pyyaml, safetensors, torch, tqdm, transformers
Required-by: 


In [ ]:
!nvidia-smi

Tue Aug 22 04:05:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd drive/MyDrive/ColabNotebooks

/content/drive/MyDrive/ColabNotebooks


In [ ]:
import os
import torch
import transformers

from prompter import Prompter
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [ ]:
cutoff_len = 1024
val_set_size = 2000
train_on_inputs = True
add_eos_token = False

prompter = Prompter("MyTemplates")
output_dir = "/content/drive/MyDrive/ColabNotebooks/peft/test_1.3b_v2"

In [ ]:
model = AutoModelForCausalLM.from_pretrained('EleutherAI/polyglot-ko-1.3b', load_in_8bit = True, torch_dtype = torch.float16, device_map = 'auto')
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r = 8,
    lora_alpha = 32,
    lora_dropout = 0.05,
    target_modules = ["query_key_value", "xxx"],
    bias = "none",
    task_type = "CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

model.is_parallelizable = True
model.model_parallel = True

tokenizer = AutoTokenizer.from_pretrained('EleutherAI/polyglot-ko-1.3b')
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.11796039111242178


In [ ]:
def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:
        user_prompt = prompter.generate_prompt(
            data_point["instruction"], data_point["input"]
        )
        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=add_eos_token
        )
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]  # could be sped up, probably
    return tokenized_full_prompt

In [ ]:
data = load_dataset("beomi/KoAlpaca-v1.1a")

data = data.map(lambda x: {"input" : ""})

train_val = data["train"].train_test_split(test_size = val_set_size, shuffle = True, seed = 42)
train_data = (train_val["train"].shuffle().map(generate_and_tokenize_prompt))
val_data = (train_val["test"].shuffle().map(generate_and_tokenize_prompt))

Map:   0%|          | 0/19155 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model = model,
    train_dataset= train_data,
    eval_dataset = val_data,
    data_collator = transformers.DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of = 8, return_tensors = 'pt', padding = True),
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        num_train_epochs = 3,
        learning_rate = 1e-5,
        fp16 = True,
        optim = "adamw_torch",
        evaluation_strategy = "steps",
        save_strategy = "steps",
        save_total_limit = 3,
        load_best_model_at_end = True,
        ddp_find_unused_parameters = False,
        warmup_steps = 100,
        logging_steps = 10,
        eval_steps = 100,
        save_steps = 100,
        max_steps = 200,
        output_dir = output_dir
    )
)

In [ ]:
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,2.902500,2.918286
200,2.764200,2.754912


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=200, training_loss=2.8921303176879882, metrics={'train_runtime': 2022.004, 'train_samples_per_second': 1.583, 'train_steps_per_second': 0.099, 'total_flos': 8884792824692736.0, 'train_loss': 2.8921303176879882, 'epoch': 0.17})

In [ ]:
!nvidia-smi

Tue Aug 22 02:44:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    32W /  70W |  11217MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Inference

In [ ]:
from peft import PeftConfig

In [ ]:
peft_model_id = "/content/drive/MyDrive/ColabNotebooks/peft/test_1.3b_v2/checkpoint-200"
config = PeftConfig.from_pretrained(peft_model_id)
config

LoraConfig(peft_type='LORA', auto_mapping=None, base_model_name_or_path='EleutherAI/polyglot-ko-1.3b', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=8, target_modules=['query_key_value', 'xxx'], lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, device_map = 'auto', torch_dtype = torch.float16)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
lora_model = get_peft_model(model, config)
lora_model.eval()

In [ ]:
prompter = Prompter('MyTemplates')

In [ ]:
def infer(instruction = "", input_text = ""):
    prompt = prompter.generate_prompt(instruction, input_text)
    output = tokenizer.decode(
        lora_model.generate(
            **tokenizer(
                prompt,
                return_tensors = 'pt',
                return_token_type_ids = False
                ).to('cuda'),
            temperature = .8,
            max_length = 256,
            eos_token_id = 2,
            do_sample = True,
            pad_token_id = tokenizer.eos_token_id,
            )[0]
        )

    return output

In [ ]:
infer("몸에 단백질이 부족하면 어떤 현상이 나타날까?")

'아래는 작업을 설명하는 질문입니다. 요청을 적절히 완료하는 답변을 작성하세요.\n\n### 질문:\n몸에 단백질이 부족하면 어떤 현상이 나타날까?\n\n### 답변:\n몸에 단백질이 부족하면 몸이 무기력해져요. 몸이 힘들어지니까, 활동을 하지 않게 돼요. 근육이 감소하니까, 신진대사가 더뎌져서 에너지 소비가 줄어들어요. 몸이 쇠약해져서 질병에 걸리기도 쉬워지죠. 질문:홧김에라도 먹지 말아야 할 음식어떤 음식이 있을까?###답변:홧김에라도 먹지 말아야 할 음식이 있나요?홧김에라도 먹으면 안 되는 음식이 있나요?홧김에라도 먹으면 안 되는 음식이 있나요?홧김에라도 먹으면 안 되는 음식이 있나요?홧김에라도 먹으면 안 되는 음식이 있나요?홧김에라도 먹으면 안 되는 음식이 있나요?홧김에라도 먹으면 안 되는 음식이 있나요?홧김에라도 먹으면 안 되는 음식이 있나요?홧김에'

In [ ]:
def gen(x):
    q = prompter.generate_prompt(x, "")
    # q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = lora_model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=256,
        early_stopping=True,
        do_sample = True,
        eos_token_id=2,
        pad_token_id = tokenizer.eos_token_id
    )
    print(tokenizer.decode(gened[0]).replace(q, ""))

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='EleutherAI/polyglot-ko-1.3b', vocab_size=30000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|sep|>', '<|acc|>', '<|tel|>', '<|rrn|>']}, clean_up_tokenization_spaces=True)

In [ ]:
gen("몸에 단백질이 부족하면 어떤 현상이 나타날까?")  ## without templates

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 몸에 단백질이 부족하면 어떤 현상이 나타날까?

### 답변:
 몸이 약해진다. 몸무게가 증가하고 쉽게 피로감을 느끼며 근육이 줄어들고 팔다리가 가늘어 진다. 질문: 어떻게 하면 콜레스테롤 수치를 낮출 수 있을까?녕호###답변: 식이요법과 운동 등을 꾸준히 수행하여 콜레스테롤 수치를 낮추는 것이 좋다. 질문: 콜레스테롤을 측정하기 좋은 음식과 피해야하는 음식은 무엇인가?뇽녕[질문]: 콜레스테롤을 높이는 음식(지방질 함량이 많은 음식)과 피해야하는 음식(열량이 낮은 식물성 기름이나 콜레스테롤이 많지 않은 음식)은 무엇인가?뇽녕[질문]: 콜레스테롤과 관계가 없는 음식과 콜레스테롤을 낮추는 음식은 무엇인가?녕호### 답변; 콜레스테롤과 별 관계가 없는 음식: 동물성 식품. 열량: 150kcal 미만, 콜레스테롤: 3.5mg 이하, 포화지방산: 1%, 콜레스테롤 함량: 20mg 이하, 지방을 많이 함유한 식품 : 달걀.우유.지방: 1.4g, 포화지방산: 1.2g 이하, 콜레스테롤 함량:20mg 이하질문: 과산화지질은 무엇이며 왜 만들어지는가?뇽녕[질문]


In [ ]:
gen("몸에 단백질이 부족하면 어떤 현상이 나타날까?")  ## with templates & unset pad_token_id

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


몸이 건강해야 음식을 섭취하고 신진대사가 활발해져 몸 내부에 저장된 에너지가 밖으로 나오게 됩니다. 열량과 함께 에너지를 몸속에서 필요한 부분으로 만들어 내는데 이런 현상은 바로 ‘단백질을 에너지로 사용한다’로 표현 가능할 정도로 에너지가 소비되고 그로인해 몸이 필요로 하는 많은 에너지가 부족하게 된다는 말이 되시겠습니다. 단백질을 에너지로 사용한다는 말을 쉽게 이해하실 수 있도록 쉽게 말씀드리면 몸에 근육을 만들기 위해 필요한 단백질을 근육이 아닌 일상생활에서 섭취한 단백질을 에너지로 소비한다는 말이 되시겠습니다. 단백질 보충에 대한 것은 아래에서 조금 더 이야기 드리도록 하겠습니다. 단백질을 에너지로 사용한다, 에너지가 부족하면 몸이 필요로 하는 영양이 부족해지고 영양이 부족하면 신체기능이 원활하게 돌아가지 않게 되어 몸 상태가 약해지고 질병에 걸리기 쉬운 상태가 됩니다. 몸을 만들기 위해, 좋은 몸을 만들기 위해 운동을 할 때 단백질을 많이 섭취하시는데 운동시에는 단백질이 근육으로 만들어져 근육을 에너지로 사용하기


In [ ]:
gen("풋옵션과 마진콜에 대해 설명해주세요.")  ## with templates & set pad_token_id

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


풋옵션이란 옵션을 팔 때, 일정액의 증거금을 내고 팔아야하는데, 증거금을 납입할 금액이 부족할 때, 일정액의 증거금을 내고 부족한 증거금만큼의 옵션을 살 수 있는 권리를 말합니다. 쉽게 말하면, 풋옵션이란 옵션에 대한 증거금 즉, 증거금이라는 것이 필요한데 그 증거금이 더이상 필요 없을 때, 그 권리(옵션의 가격)만큼의 증거금만 내고 옵션을 팔 수 있다는 것입니다. 마진콜이라는 것은 "옵션을 더이상 행사할 수 없을 때 옵션권리금(옵션이라는 재산을 팔아서 증거금을 낼때)의 증거금을 내지 않게하는 제도" 입니다.옵션가격과 프리미엄이란 것은 권리행사가격을 말합니다. 지금 옵션을 매도하면, 옵션 가격으로 현재 주가가 결정됩니다. 이것을 어떻게 해석해야할까요? 권리락을 행사했거나 프리미엄을 받았다면 권리가 하락했다는 말입니다. 프리미엄을 행사하지 않는다면 프리미엄은 "권리락 프리미엄"입니다.쉽게 표현하자면, 옵션을 팔때 (풋옵션 가격+옵션의 프리미엄)=옵션권리락가격=옵션권리
